In [ ]:
Pet1.ipynb
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers

# -----------------------------
# 1. 손상 이미지 확실히 제거
def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.load()
            if img.mode not in ['RGB', 'RGBA', 'L']:  # 3채널, 4채널, 흑백만 허용
                print(f"삭제 대상(채널 문제): {file_path}, 모드: {img.mode}")
                return False
        return True
    except Exception as e:
        print(f"삭제 대상(불러오기 실패): {file_path}, 이유: {e}")
        return False

for category in ["Cat", "Dog"]:
    folder_path = os.path.join("PetImages", category)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        if not is_valid_image(fpath):
            os.remove(fpath)

# -----------------------------
# 2. 경로 및 하이퍼파라미터
data_dir = pathlib.Path('./PetImages')
img_height = 180
img_width = 180
batch_size = 64

# -----------------------------
# 3. 손상 제거 후 "다시" 데이터 로딩
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    str(data_dir),
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# 클래스 확인
class_names = full_ds.class_names
num_classes = len(class_names)
print("클래스:", class_names)

# -----------------------------
# 4. train/val 분할
AUTOTUNE = tf.data.AUTOTUNE
total_batches = tf.data.experimental.cardinality(full_ds).numpy()
train_size = int(0.8 * total_batches)
train_ds = full_ds.take(train_size).shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = full_ds.skip(train_size).prefetch(buffer_size=AUTOTUNE)

# -----------------------------
# 5. 모델 구성 및 학습
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

model = tf.keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# 6. 학습 실행
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

# -----------------------------
# 7. 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

# -----------------------------
# 8. 모델 저장
os.makedirs('./model', exist_ok=True)
model.save('./model/cat_dog_classification_model.h5')
print("모델이 저장되었습니다!")
